In [1]:
from __future__ import print_function, division

import thinkbayes2

from sympy import symbols

In [2]:
p, q, s, t1, t2 = symbols('p q s t1 t2')

In [3]:
a, b, c, d, e, f, g, h = symbols('a b c d e f g h')

In [4]:
a =   q   *   p   *   s
b =   q   *   p   * (1-s)
c =   q   * (1-p) *   t1
d =   q   * (1-p) * (1-t1)
e = (1-q) *   p   *   s
f = (1-q) *   p   * (1-s)
g = (1-q) * (1-p) *   t2
h = (1-q) * (1-p) * (1-t2)

In [5]:
pmf_t = thinkbayes2.Pmf({t1:q, t2:1-q})
pmf_t.Mean().simplify()

1.0*q*t1 - 1.0*t2*(q - 1)

In [6]:
d1 = dict(q=0.5, p=0.1, s=0.5, t1=0.2, t2=0.8)
pmf_t.Mean().evalf(subs=d1)

0.500000000000000

In [7]:
d2 = dict(q=0.75, p=0.1, s=0.5, t1=0.4, t2=0.8)
pmf_t.Mean().evalf(subs=d2)

0.500000000000000

In [8]:
def prob(yes, no):
    return yes / (yes + no)

In [9]:
res = prob(a+e, c+g)
res.simplify()

p*s/(-p*q*t1 + p*q*t2 + p*s - p*t2 + q*t1 - q*t2 + t2)

In [10]:
res.evalf(subs=d1)

0.100000000000000

In [11]:
res.evalf(subs=d2)

0.100000000000000

Scenario 2

In [12]:
p1 = prob(a, c)
p1.simplify()

p*s/(p*s - t1*(p - 1))

In [13]:
p1.evalf(subs=d1)

0.217391304347826

In [14]:
p2 = prob(e, g)
p2.simplify()

p*s/(p*s - t2*(p - 1))

In [15]:
p2.evalf(subs=d1)

0.0649350649350649

In [16]:
pmf = thinkbayes2.Pmf([p1, p2])

In [17]:
pmf.Mean().simplify()

0.5*p*s*(2*p*s - t1*(p - 1) - t2*(p - 1))/((p*s - t1*(p - 1))*(p*s - t2*(p - 1)))

In [18]:
pmf.Mean().evalf(subs=d1)

0.141163184641446

In [19]:
p1.evalf(subs=d2), p2.evalf(subs=d2), pmf.Mean().evalf(subs=d2)

(0.121951219512195, 0.0649350649350649, 0.0934431422236300)

Scenario 3

In [20]:
def update(pmf):
    post = pmf.Copy()
    post[p1] *= (a + c) / q
    post[p2] *= (e + g) / (1-q)
    post.Normalize()
    return post

In [21]:
post = update(pmf)
post[p1].simplify()

(-0.5*p*s + 0.5*t1*(p - 1))/(-1.0*p*s + 0.5*t1*(p - 1) + 0.5*t2*(p - 1))

In [22]:
post.Mean().simplify()

-1.0*p*s/(-1.0*p*s + 0.5*t1*(p - 1) + 0.5*t2*(p - 1))

In [23]:
post.Mean().evalf(subs=d1)

0.100000000000000

In [24]:
post.Mean().evalf(subs=d2)

0.0847457627118644

In [25]:
post2 = update(post)

In [26]:
post2.Mean().simplify()

1.0*p*s*(2*p*s - t1*(p - 1) - t2*(p - 1))/((p*s - t1*(p - 1))**2 + (p*s - t2*(p - 1))**2)

In [27]:
post2.Mean().evalf(subs=d1)

0.0774233508826262

In [28]:
post2.Mean().evalf(subs=d2)

0.0775295663600526

In [29]:
post3 = update(post2)
post3.Mean().evalf(subs=d1)

0.0688926818860678

In [30]:
post3.Mean().evalf(subs=d2)

0.0724135699794844